# Scrape Data from shopee with selenium and beatifulsoup

### 1. Pip install library

In [ ]:
!pip install selenium
!pip install bs4

### 2. Import library

In [6]:
import pandas as pd
from selenium import webdriver
import bs4
import time
import re
from selenium.webdriver.common.keys import Keys

### 3. Selenium: Control and web page preparation

3.1 Open browser

In [ ]:
from selenium import webdriver

url = "https://shopee.co.th/"
driver = webdriver.Chrome(executable_path = r'C:\Users\User\Desktop\Python_shopee\chromedriver')
driver.get(url)

3.2 Select element on website (select thai_button)

In [8]:
thai_button = driver.find_element("xpath",'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

3.3 Close advertising

In [9]:
close_button = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_button.click()

3.4 Search bar for searching keyword

In [ ]:
keyword ="หม้อทอดไร้น้ำมัน"

search = driver.find_element("xpath",'/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys(keyword)

3.5 Use keyboard press ENTER

In [ ]:
from selenium.webdriver.common.keys import Keys
search.send_keys(Keys.ENTER)

Zoom out

In [ ]:
driver.execute_script("document.body.style.zoom='10%'")
driver.execute_script("document.body.style.zoom='100%'")

3.6 Next page

In [ ]:
next_button = driver.find_element("xpath","/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[@class='shopee-icon-button shopee-icon-button--right ']")
next_button.click()

### 4. Beautiful soup: Scraping process

4.1 เก็บข้อมูลจากหน้าเว็บ <br>
- ใช้ดึง html ของ browser ที่เปิดอยู่ออกมา ตัวที่ดึงได้ในขณะที่คำสั่งทำงาน <br>
- ถ้าก็บไว้ในตัวแปร data แล้ว สามารถปิดหน้าเว็บได้เลย (ไม่ได้เชื่อมกันแล้ว)

In [ ]:
import bs4

data = driver.page_source
soup = bs4.BeautifulSoup(data)
box_products = soup.select(".row>div") #css selector

check amoung of data : 60 datas

In [ ]:
len(box_products)
product = box_products[1] #แทนกล่องสินค้าแต่ละกล่อง

4.2 select NAME data (~select_one)

In [ ]:
product.select_one("div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.dpiR4u > div.FDn--\+ > div").text.strip()

4.3 select PRICE data

In [ ]:
try:
    p = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.hpDKMN > div.vioxXd.rVLWG6").text.strip()
    if p == " ":
        print(1/0)
except:
    p = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.hpDKMN > div").text.strip()
p = p.replace(",","").replace("฿","").replace(" ", "")
try:
    start_price = float(p)
except:
    price = p.split("-")
    list_x = []
    for i in price:
        x = str(float(i))
        list_x.append(x)
    start_price = list_x[0]+"-"+list_x[1]
start_price

4.4 select SALES data<br>
แปลงหน่วย พัน และ ล้าน เป็นจำนวนตัวเลข 000 , 000000

In [ ]:
import re

sales = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.ZnrnMl").text.strip()
if '.' in sales:
    sales = sales.replace("พัน","00").replace("ล้าน","00000").replace(".","")
else:
    sales = sales.replace("พัน","000").replace("ล้าน","000000")
try:
    sales = float(re.sub('\D',' ',sales))
except:
    sales = 0
sales

4.5 select ORIGIN OF PROVINCE data

In [ ]:
product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.zGGwiV").text.strip()

4.6 save to EXCEL file

In [ ]:
df = pd.DataFrame(records, columns=['url','itemname','Price range','Sales','Province'])
df.to_excel(rf'C:\Users\User\Desktop\Python_shopee\shopee_{search_keyword}.xlsx')

### 5. Fully coding (Automatic) </br>
 - ระบุ keyword
 - จำนวนหน้า

In [ ]:
import pandas as pd
from selenium import webdriver
import bs4
import time
import re
from selenium.webdriver.common.keys import Keys

search_keyword = "หม้อทอดไร้น้ำมัน"
pages = 1

driver = webdriver.Chrome(executable_path = r'C:\Users\User\Desktop\Python_shopee\chromedriver')
driver.get("https://shopee.co.th/")
thai_button = driver.find_element("xpath",'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
close_ads = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_ads.click()
search = driver.find_element("xpath",'/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys(search_keyword)
search.send_keys(Keys.ENTER)

#driver.execute_script("document.body.style.zoom='10%'")
#driver.execute_script("document.body.style.zoom='100%'")

#time.sleep(5)
records = []
for i in range (pages) :
    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(5)
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    box_products = soup.select(".row>div")
    for product in box_products:
        names = product.select_one("div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.dpiR4u > div.FDn--\+ > div").text.strip()
        
        try:
            p = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.hpDKMN > div.vioxXd.rVLWG6").text.strip()
            if p == " ":
                print(1/0)
        except:
            p = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.hpDKMN > div").text.strip()
        p = p.replace(",","").replace("฿","").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.split("-")
            list_x = []
            for i in price:
                x = str(float(i))
                list_x.append(x)
            start_price = list_x[0]+"-"+list_x[1]

        sales = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.ZnrnMl").text.strip()
        if '.' in sales:
            sales = sales.replace("พัน","00").replace("ล้าน","00000").replace(".","")
        else:
            sales = sales.replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D',' ',sales))
        except:
            sales = 0
        
        provinces = product.select_one("#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a > div > div > div.KMyn8J > div.zGGwiV").text.strip()
        
        product.select_one('#main > div > div.dYFPlI > div > div > div.ZgwlcK > div.shopee-search-item-result > div.row.shopee-search-item-result__items > div > a')
        for link in product.find_all('a', href=True):
            link = link['href']
        url = "https://shopee.co.th"+link
        
        records.append([url,names,start_price,sales,provinces])
        
    time.sleep(5)
    driver.execute_script("document.body.style.zoom='100%'")
    #next_button = driver.find_element("xpath","/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[8]")
    next_button = driver.find_element("xpath","/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[@class='shopee-icon-button shopee-icon-button--right ']")
    next_button.click()
    time.sleep(5)

df = pd.DataFrame(records, columns=['url','itemname','Price range','Sales','Province'])
df.to_excel(rf'C:\Users\User\Desktop\Python_shopee\shopee_{search_keyword}.xlsx')